## CONFIGURE PRIMARY CLUSTER

In [1]:
%%bash
gcloud auth login
export VAULT_ADDR=$(terraform output -raw cluster_primary_fqdn_8200)
vault_token_primary=$(terraform output -raw cluster_primary_read_vault_token)
# Execute the command stored in $vault_token
export VAULT_TOKEN=$(eval "$vault_token_primary")


vault write -f sys/replication/dr/primary/enable primary_cluster_addr=$(terraform output -raw cluster_primary_fqdn_8201)
sleep 5

vault write -format=json sys/replication/dr/primary/secondary-token id=dr12 | jq -r .wrap_info.token > dr_token.txt



Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8087%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=0BL7TN4mdb5pbDANUcxr9ee5gXYmSp&access_type=offline&code_challenge=ex3ej3i6Xd52V-lipkhJyQOliEcwX8PFFzKZP5NPdhI&code_challenge_method=S256


You are now logged in as [jose.merchan@hashicorp.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID
WARNING! The following warnings were returned from Vault:

  * This cluster is being enabled as a primary for replication. Vault will 

## CONFIGURE SECONDARY CLUSTER

In [2]:
%%bash

export VAULT_ADDR=$(terraform output -raw cluster_dr_fqdn_8200)
vault_token_dr=$(terraform output -raw cluster_dr_read_vault_token)
# Execute the command stored in $vault_token
export VAULT_TOKEN=$(eval "$vault_token_dr")

vault write sys/replication/dr/secondary/enable  \
    primary_api_addr=$(terraform output -raw cluster_primary_fqdn_8200) \
    token=$(cat dr_token.txt)

WARNING! The following warnings were returned from Vault:

  * Vault has successfully found secondary information; it may take a while to
  perform setup tasks. Vault will be unavailable until these tasks and initial
  sync complete.



In [3]:
%%bash

export VAULT_ADDR=$(terraform output -raw cluster_primary_fqdn_8200)
vault_token_primary=$(terraform output -raw cluster_primary_read_vault_token)
export VAULT_TOKEN=$(eval "$vault_token_primary")

vault policy write dr-secondary-promotion - <<EOF
path "sys/replication/dr/secondary/promote" {
  capabilities = [ "update" ]
}

# To update the primary to connect
path "sys/replication/dr/secondary/update-primary" {
    capabilities = [ "update" ]
}

# Only if using integrated storage (raft) as the storage backend
# To read the current autopilot status
path "sys/storage/raft/autopilot/state" {
    capabilities = [ "update" , "read" ]
}

path "sys/storage/raft/*" {
    capabilities = [ "update" , "read", "create", "delete", "patch", "sudo" ]
}
EOF

vault write auth/token/roles/failover-handler \
    allowed_policies=dr-secondary-promotion \
    orphan=true \
    renewable=false \
    token_type=batch


vault token create -role=failover-handler -ttl=8h

vault read sys/storage/raft/autopilot/state

Success! Uploaded policy: dr-secondary-promotion
Success! Data written to: auth/token/roles/failover-handler
Key                  Value
---                  -----
token                hvb.AAAAAQL0-v-OEZ4NPNE8Li48a0KvcPwMfcmfsrOielB2tcKoUi5hmF9AgdUYFAiduUwwv4RqRpZu0SD0OnOTEA6zX1qMovH72RoWCtSx229Pp4O2R0ac4t6kNRx0y2xik2PHGPnBTsNxyFaqgTDUL2o4Mnzz3iJ2eEIiyLc_Iw-6yzU66tMwwKGy9BRE8-XINWMa2fbeiA
token_accessor       n/a
token_duration       8h
token_renewable      false
token_policies       ["default" "dr-secondary-promotion"]
identity_policies    []
policies             ["default" "dr-secondary-promotion"]
Key                             Value
---                             -----
failure_tolerance               2
healthy                         true
leader                          vault-primary-0
optimistic_failure_tolerance    2
servers                         map[vault-primary-0:map[address:vault-primary-0.vault-primary-internal:8201 healthy:true id:vault-primary-0 last_contact:0s last_ind

In [8]:
%%bash
export VAULT_ADDR=$(terraform output -raw cluster_primary_fqdn_8200)
vault_token_primary=$(terraform output -raw cluster_primary_read_vault_token)
export VAULT_TOKEN=$(eval "$vault_token_primary")

vault token create -role=failover-handler -ttl=8h -format=json | jq -r .auth.client_token > dr_batch_token.txt

In [9]:
%%bash
export VAULT_ADDR=$(terraform output -raw cluster_dr_fqdn_8200)
export VAULT_TOKEN=$(cat dr_batch_token.txt)

vault read sys/storage/raft/autopilot/state

Key                             Value
---                             -----
failure_tolerance               2
healthy                         true
leader                          vault-dr-2
optimistic_failure_tolerance    2
servers                         map[vault-dr-0:map[address:vault-dr-0.vault-dr-internal:8201 healthy:true id:vault-dr-0 last_contact:616.828535ms last_index:2173 last_term:4 name:vault-dr-0 node_status:alive node_type:voter stable_since:2024-11-18T10:16:25.601846214Z status:voter upgrade_version:1.18.1 version:1.18.1] vault-dr-1:map[address:vault-dr-1.vault-dr-internal:8201 healthy:true id:vault-dr-1 last_contact:555.515577ms last_index:2173 last_term:4 name:vault-dr-1 node_status:alive node_type:voter stable_since:2024-11-18T10:16:25.601846214Z status:voter upgrade_version:1.18.1 version:1.18.1] vault-dr-2:map[address:vault-dr-2.vault-dr-internal:8201 healthy:true id:vault-dr-2 last_contact:0s last_index:2178 last_term:4 name:vault-dr-2 node_status:alive node_type: